# PRÁCTICA: Mejorando la Relevancia en un Sistema RAG

## 1. Preparación del entorno

Instalamos las librerías necesarias:

In [ ]:
# pip install chromadb sentence-transformers


Importamos los módulos principales:

In [ ]:
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer, util


## 2. Crear una mini base de conocimiento

Vamos a simular una base de datos con textos sobre tecnología, astronomía y animales.

## 3. Crear la base vectorial (ChromaDB)

Creamos una colección donde almacenaremos los embeddings de los textos.

## 4. Realizar una búsqueda semántica básica

Consultamos algo relacionado con animales domésticos:

## 5. Optimización 1: Chunking Estratégico

Supongamos que tenemos textos largos. Vamos a dividirlos en chunks lógicos con solapamiento (simulado).

## 6. Optimización 2: Re-ranking

Vamos a mejorar la búsqueda semántica seleccionando los resultados más relevantes mediante una segunda evaluación (re-ranking).

## 7. Optimización 3: Búsqueda híbrida

Combinaremos búsqueda por palabra clave + semántica.

## 8. Evaluación de la mejora

Creamos una pequeña tabla de comparación:

| **Técnica aplicada** | **Relevancia esperada** | **Latencia estimada** | **Comentario breve** |
|-----------------------|-------------------------|------------------------|-----------------------|
| Búsqueda básica       | Media                   | Rápida                 | A veces mezcla temas |
| Re-ranking            | Alta                    | Media                  | Resultados más precisos |
| Búsqueda híbrida      | Alta                    | Media-Alta             | Captura contexto + palabras clave |


## 9. Discusión 

- ¿Cuál técnica dio mejores resultados en relevancia?

- ¿Qué técnica aumentó la latencia, y por qué?

- ¿Cómo combinarías re-ranking + caching para un chatbot real?

- ¿Qué ocurre si el corpus tiene información ambigua?

## Extensión 

Implementa un caché simple que guarde los últimos resultados de consulta.
Tip: Usa un diccionario {consulta: resultados} y revisa si la consulta ya existe antes de buscar otra vez.